# Phase 1: Naive Monte Carlo Simulation

This notebook implements the naive Monte Carlo approach for estimating transition probabilities in the double well potential system.

## Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from tqdm import tqdm
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

# Import our modules
from potentials import double_well_potential, double_well_gradient, create_potential_grid
from integrators import euler_maruyama_step, euler_maruyama_trajectory
from detectors import in_region_A, in_region_B, check_absorption, simulate_until_absorption
from monte_carlo import run_naive_mc, print_results_summary
from utils import plot_potential_field, plot_trajectory, plot_absorption_time_histogram, save_figure

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True

Matplotlib is building the font cache; this may take a moment.


ImportError: attempted relative import with no known parent package

## 1. System Visualization

First, let's visualize the double well potential and the regions A and B.

In [ ]:
# Plot the potential field
fig, ax = plot_potential_field(omega=1.0, show_regions=True, r_A=0.2, r_B=0.2)
plt.show()

# Save the figure
save_figure(fig, 'double_well_potential')

## 2. Test Single Trajectory

Let's simulate a single trajectory to understand the dynamics.

In [ ]:
# Parameters for single trajectory
x0, y0 = -1.0, 0.0  # Start near left minimum
dt = 0.01
beta_inv = 0.5      # Moderate temperature
max_steps = 5000
omega = 1.0

# Simulate until absorption
trajectory, absorption_state, absorption_time = simulate_until_absorption(
    x0, y0, dt, beta_inv, max_steps, omega
)

print(f"Trajectory length: {len(trajectory)} steps")
print(f"Absorption state: {absorption_state} (0=none, 1=A, 2=B)")
print(f"Absorption time: {absorption_time} steps")

# Plot the trajectory
fig, ax = plot_trajectory(trajectory, absorption_state)
plt.show()

save_figure(fig, 'single_trajectory')

## 3. Naive Monte Carlo Simulation

Now let's run multiple simulations to estimate the transition probability $p = \mathbb{P}(\tau_B < \tau_A)$.

In [ ]:
# Parameters for Monte Carlo simulation
n_simulations = 1000
x0, y0 = -1.0, 0.0  # Start in left well
dt = 0.01
beta_inv = 0.5      # Moderate temperature
max_steps = 10000
omega = 1.0
seed = 42

print("Running naive Monte Carlo simulation...")
results = run_naive_mc(n_simulations, x0, y0, dt, beta_inv, max_steps, 
                       omega=omega, seed=seed)

print_results_summary(results)

## 4. Visualization of Results

Let's visualize the distribution of absorption times.

In [ ]:
# Plot absorption time histogram
fig, ax = plot_absorption_time_histogram(results)
plt.show()

save_figure(fig, 'absorption_time_histogram')

## 5. Study of Time Step Δt

Let's investigate the effect of different time steps on the results.

In [ ]:
dt_values = [0.001, 0.005, 0.01, 0.05]
n_simulations_dt = 500  # Fewer simulations for faster computation

dt_results = {}

for dt in tqdm(dt_values, desc="Testing different Δt"):
    results_dt = run_naive_mc(n_simulations_dt, x0, y0, dt, beta_inv, max_steps, 
                             omega=omega, seed=seed)
    dt_results[dt] = results_dt

# Print results for comparison
print("\nComparison of results for different time steps:")
print("=" * 60)
for dt, res in dt_results.items():
    print(f"Δt = {dt:.4f}: p = {res['p_estimate']:.6f} "
          f"CI95: [{res['ci_95'][0]:.6f}, {res['ci_95'][1]:.6f}]")
    print(f"         B transitions: {res['n_B']}/{n_simulations_dt}")

## 6. Study of Temperature (β⁻¹)

Let's investigate how temperature affects the transition probability.

In [ ]:
beta_inv_values = [0.1, 0.3, 0.5, 0.7, 1.0]
n_simulations_temp = 500

temp_results = {}

for beta_inv in tqdm(beta_inv_values, desc="Testing different temperatures"):
    results_temp = run_naive_mc(n_simulations_temp, x0, y0, dt, beta_inv, max_steps, 
                               omega=omega, seed=seed)
    temp_results[beta_inv] = results_temp

# Print results for comparison
print("\nComparison of results for different temperatures:")
print("=" * 70)
for beta_inv, res in temp_results.items():
    print(f"β⁻¹ = {beta_inv:.2f}: p = {res['p_estimate']:.6f} "
          f"CI95: [{res['ci_95'][0]:.6f}, {res['ci_95'][1]:.6f}]")
    print(f"           B transitions: {res['n_B']}/{n_simulations_temp}")

## 7. Summary and Conclusions

Let's create a summary table of our findings.

In [ ]:
print("SUMMARY OF NAIVE MONTE CARLO RESULTS")
print("=" * 50)
print(f"Base parameters: ω={omega}, x0=({x0},{y0}), max_steps={max_steps}")
print()

# Main result
print("Main simulation (n=1000, dt=0.01, β⁻¹=0.5):")
print(f"  p = {results['p_estimate']:.6f}")
print(f"  95% CI: [{results['ci_95'][0]:.6f}, {results['ci_95'][1]:.6f}]")
print()

# Time step study
print("Time step study (n=500, β⁻¹=0.5):")
for dt, res in dt_results.items():
    print(f"  Δt={dt:.4f}: p={res['p_estimate']:.6f}")
print()

# Temperature study
print("Temperature study (n=500, dt=0.01):")
for beta_inv, res in temp_results.items():
    print(f"  β⁻¹={beta_inv:.2f}: p={res['p_estimate']:.6f}")
print()

print("Conclusions:")
print("- The naive MC method provides reliable estimates for measurable probabilities")
print("- The method becomes inefficient for very rare events (p < 10^-3)")
print("- This motivates the need for more advanced methods like AMS")

## 8. Save Results

Save the simulation results for future reference.

In [ ]:
import json
import pickle

# Create results directory
os.makedirs('../experiments/results', exist_ok=True)

# Save main results
with open('../experiments/results/naive_mc_main.json', 'w') as f:
    json.dump({
        'parameters': {
            'n_simulations': n_simulations,
            'x0': x0, 'y0': y0,
            'dt': dt, 'beta_inv': beta_inv,
            'max_steps': max_steps,
            'omega': omega,
            'seed': seed
        },
        'results': {
            'p_estimate': results['p_estimate'],
            'ci_95': results['ci_95'],
            'n_A': results['n_A'],
            'n_B': results['n_B'],
            'n_unabsorbed': results['n_unabsorbed']
        }
    }, f, indent=2)

# Save full results with pickle
with open('../experiments/results/naive_mc_full.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Results saved to experiments/results/")
print("Phase 1 implementation complete!")